In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
import os
import pandas as pd

CSV_PATH = "Metadata.csv"
TRAIN_IMG_DIR = "dataset/images"
VAL_IMG_DIR = "validation/images"

df = pd.read_csv(CSV_PATH, encoding='cp949')
df = df[df["road_grade"].notnull()]
df['road_grade'] = df['road_grade'].astype(int)

train_df = df[df['FileName'].apply(lambda x: os.path.exists(os.path.join(TRAIN_IMG_DIR, x)))]
train_df = train_df.copy()
train_df['full_path'] = train_df['FileName'].apply(lambda x: os.path.join(TRAIN_IMG_DIR, x))

val_df = df[df['FileName'].apply(lambda x: os.path.exists(os.path.join(VAL_IMG_DIR, x)))]
val_df = val_df.copy()
val_df['full_path'] = val_df['FileName'].apply(lambda x: os.path.join(VAL_IMG_DIR, x))

NUM_CLASSES = df['road_grade'].nunique()
print(f"클래스 수: {NUM_CLASSES}")
print(f"Train: {len(train_df)}, Val: {len(val_df)}")

클래스 수: 3
Train: 209642, Val: 26204


In [3]:
from torch.utils.data import Dataset
from PIL import Image

class RoadDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = Image.open(self.df.loc[idx, 'full_path']).convert("RGB")
        label = int(self.df.loc[idx, 'road_grade'])

        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
from torch.utils.data import DataLoader
from torchvision import transforms

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(384, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])
val_tf = transforms.Compose([
    transforms.Resize(420), 
    transforms.CenterCrop(384),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

train_dataset = RoadDataset(train_df, transform=train_tf)
val_dataset   = RoadDataset(val_df,   transform=val_tf)

BATCH_SIZE = 16

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=6, pin_memory=True, persistent_workers=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=6, pin_memory=True, persistent_workers=True)


In [5]:
import torch
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights
from tqdm import tqdm
from PIL import ImageFile
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
ImageFile.LOAD_TRUNCATED_IMAGES = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#1차 수정
import timm
model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.05).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
EPOCHS = 10
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = torch.cuda.amp.GradScaler()

best_acc = 0.0

# 학습 루프
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1} Validation"):
            images, labels = images.to(device), labels.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total * 100
    print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}, Val Acc: {acc:.2f}%")

    scheduler.step()
    
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_cls.pth")

/home/user22313548/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_8687/1093001931.py:27: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1 Training:   0%|                                                                       | 0/13103 [00:00<?, ?it/s]/tmp/ipykernel_8687/1093001931.py:50: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1 Validation:   0%|                                                                      | 0/1638 [00:00<?, ?it/s]/tmp/ipykernel_8687/1093001931.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. 

[Epoch 1] Loss: 7006.8080, Val Acc: 83.20%


Epoch 2 Validation: 100%|███████████████████████████████████████████████████████████| 1638/1638 [03:49<00:00,  7.13it/s]


[Epoch 2] Loss: 6391.0628, Val Acc: 83.96%


Epoch 3 Validation: 100%|███████████████████████████████████████████████████████████| 1638/1638 [02:53<00:00,  9.45it/s]


...

[Epoch 8] Loss: 5444.8282, Val Acc: 85.78%


Epoch 9 Validation: 100%|███████████████████████████████████████████████████████████| 1638/1638 [02:54<00:00,  9.40it/s]


[Epoch 9] Loss: 5327.3113, Val Acc: 85.70%


Epoch 10 Validation: 100%|██████████████████████████████████████████████████████████| 1638/1638 [02:52<00:00,  9.50it/s]

[Epoch 10] Loss: 5268.0545, Val Acc: 85.67%
